In [3]:
import numpy as np
import pandas as pd

In [4]:
import json

In [49]:
from importlib import reload 
import bright_future_base
reload(bright_future_base)

<module 'bright_future_base' from '\\\\wsl$\\Ubuntu-20.04\\home\\syoro\\bright-future\\src\\bright_future_base.py'>

In [50]:
df_authors = bright_future_base.load_df_authors()

Hypothetical: Say Tim has won the "Kraska Krown", Sam has won the "Madden Memorial (RIP Sam) Award". And, they have each won the "6S079 Prof Award", in different years.

In [16]:
award_winners = bright_future_base.load_award_winners()
award_winners

{'Kraska Krown': {2021: 'Tim Kraska'},
 'Madden Memorial (RIP Sam) Award': {2020: 'Samuel Madden'},
 '6.S079 Prof Award': {2021: 'Tim Kraska', 2020: 'Samuel Madden'}}

In [17]:
# TODO move this filtering into helper files, so that users don't have to do it themselves.
# smth like filter_authors(name='Sam', year='2020')
# filter_for_award_winner( (2020:'Sam') ) 
madden_award_winner = df_authors[(df_authors['name'] == 'Samuel Madden') & (df_authors['year']==2020)]
madden_award_winner

,name,dept,area,count,adjustedcount,year,title
0,Samuel Madden,Massachusetts Institute of Technology,aaai,1,0.1,2020,AI
3,Samuel Madden,Massachusetts Institute of Technology,eccv,1,0.11111,2020,Vision
18,Samuel Madden,Massachusetts Institute of Technology,icde,1,0.2,2020,DB
24,Samuel Madden,Massachusetts Institute of Technology,mobisys,1,0.1,2020,Mobile
28,Samuel Madden,Massachusetts Institute of Technology,oopsla,1,0.25,2020,PL
48,Samuel Madden,Massachusetts Institute of Technology,sigmod,5,1.0611,2020,DB
69,Samuel Madden,Massachusetts Institute of Technology,vldb,1,0.25,2020,DB


In [103]:
count_works_by_name('Samuel Madden')

{'AI': 1,
 'DB': 85,
 'HCI': 1,
 'ML': 1,
 'Mobile': 8,
 'Networks': 3,
 'PL': 2,
 'SE': 1,
 'Systems': 6,
 'Vision': 4,
 'Web & IR': 1}

How well does a diff person's research profile (hypothetically, Tim's cuz im too lazy to load data) stick up to the award winner?

In [35]:
# TODO option to use raw or adjusted counts

In [8]:
my_authors = df_authors[(df_authors['name'] == 'Tim Kraska') & (df_authors['year']==(2020))]
my_authors

,name,dept,area,count,adjustedcount,year,title
76,Tim Kraska,Massachusetts Institute of Technology,icde,1,0.33333,2020,DB
78,Tim Kraska,Massachusetts Institute of Technology,mobisys,1,0.1,2020,Mobile
90,Tim Kraska,Massachusetts Institute of Technology,sigmod,7,1.3444,2020,DB
101,Tim Kraska,Massachusetts Institute of Technology,vldb,3,0.54167,2020,DB


In [56]:
# TODO move this into helper file
def group_works_by_title(df_authors):
    return df_authors.groupby('title').sum()

# TODO count or adjusted?
# TODO move
def count_works_by_title(df_authors):
    grouped = group_works_by_title(df_authors)
    return grouped['count'].to_dict()

def get_works_by_author(name):
    return df_authors[(df_authors['name'] == name) ]

def count_works_by_name(name):
    return count_works_by_title(get_works_by_author(name))

In [45]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [48]:
# Group works, (optionally filter years), then run similarity.
# Compute similarity by title or area? should use option
def similarity_author_works(name_1, name_2, year=None, metric='cosine', adjusted = False):
    works_1 = df_authors[(df_authors['name'] == name_1) ]
    works_2 = df_authors[(df_authors['name'] == name_2) ]
    
    works_by_title_1 = count_works_by_title(works_1)
    works_by_title_2 = count_works_by_title(works_2)
    v = DictVectorizer()
    works_stacked = [works_by_title_1, works_by_title_2]
    works_encoded = v.fit_transform(works_stacked)
    print(works_by_title_1, works_by_title_2)
    return cosine_similarity(works_encoded)

similarity_author_works('Samuel Madden', 'Tim Kraska')

{'AI': 1, 'DB': 85, 'HCI': 1, 'ML': 1, 'Mobile': 8, 'Networks': 3, 'PL': 2, 'SE': 1, 'Systems': 6, 'Vision': 4, 'Web & IR': 1} {'DB': 54, 'HCI': 3, 'ML': 2, 'Mobile': 1, 'Systems': 1, 'Web & IR': 1}


array([[1.        , 0.99245207],
       [0.99245207, 1.        ]])

So, Tim's research profile is quite similar to the Madden award winner!

In [62]:
w1= count_works_by_name('Tim Kraska')
w2=count_works_by_name('Samuel Madden')
w3 = count_works_by_name('David R. Karger')

works_stacked = [w1, w2, w3]
v = DictVectorizer()
works_encoded = v.fit_transform(works_stacked)


In [63]:
works_encoded

<3x17 sparse matrix of type '<class 'numpy.float64'>'
	with 32 stored elements in Compressed Sparse Row format>

In [65]:
from sklearn.linear_model import SGDClassifier

In [80]:
import numpy as np

In [99]:
lin_sgd = SGDClassifier(loss='log')
lin_sgd.fit(works_encoded[0:3], [0, 1, 0])

SGDClassifier(loss='log')

In [102]:
lin_sgd.predict_proba(works_encoded[0])

array([[1., 0.]])